In [1]:
import os
%pwd

'd:\\Developments\\Python\\Fast Food Classification\\Fast-Food-Classification\\research'

In [2]:
os.chdir("../")

In [3]:
%pwd

'd:\\Developments\\Python\\Fast Food Classification\\Fast-Food-Classification'

In [4]:
import tensorflow as tf

In [5]:
model = tf.keras.models.load_model("artifacts/training/model.keras")


In [6]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    validation_data:Path
    test_data: Path
    all_params: dict
    params_image_size: list
    params_batch_size: int

In [7]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [8]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_validation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.keras",
            training_data="artifacts/data_ingestion/Fast Food Classification V2/Train",
            validation_data="artifacts/data_ingestion/Fast Food Classification V2/Valid",
            test_data="artifacts/data_ingestion/Fast Food Classification V2/Test",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [9]:
from urllib.parse import urlparse

In [10]:
import tensorflow as tf
from pathlib import Path

class Evaluation:
    def __init__(self, config):
        self.config = config

    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale=1./255
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.validation_data,
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        self._save_json(path=Path("scores.json"), data=scores)

    @staticmethod
    def _save_json(path: Path, data: dict):
        import json
        with open(path, 'w') as f:
            json.dump(data, f)


In [11]:
# class Evaluation:
#     def __init__(self, config: EvaluationConfig):
#         self.config = config

    
#     def _valid_generator(self):

#         datagenerator_kwargs = dict(
#             rescale = 1./255,
#             validation_split=0.30
#         )

#         dataflow_kwargs = dict(
#             target_size=self.config.params_image_size[:-1],
#             batch_size=self.config.params_batch_size,
#             interpolation="bilinear"
#         )

#         valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
#             **datagenerator_kwargs
#         )

#         self.valid_generator = valid_datagenerator.flow_from_directory(
#             directory=self.config.training_data,
#             subset="validation",
#             shuffle=False,
#             **dataflow_kwargs
#         )

    
#     @staticmethod
#     def load_model(path: Path) -> tf.keras.Model:
#         return tf.keras.models.load_model(path)
    

#     def evaluation(self):
#         self.model = self.load_model(self.config.path_of_model)
#         self._valid_generator()
#         self.score = model.evaluate(self.valid_generator)

    
#     def save_score(self):
#         scores = {"loss": self.score[0], "accuracy": self.score[1]}
#         save_json(path=Path("scores.json"), data=scores)


In [12]:
try:
    config = ConfigurationManager()
    val_config = config.get_validation_config()
    evaluation = Evaluation(val_config)
    evaluation.evaluation()
    evaluation.save_score()

except Exception as e:
   raise e

[2024-07-06 22:47:13,131: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-07-06 22:47:13,133: INFO: common: yaml file: params.yaml loaded successfully]
[2024-07-06 22:47:13,134: INFO: common: created directory at: artifacts]
Found 3500 images belonging to 10 classes.


c:\Users\Lakshitha\.conda\envs\fastfood\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


 62/219 ━━━━━━━━━━━━━━━━━━━━ 4:17 2s/step - accuracy: 0.1365 - loss: 14.6036

KeyboardInterrupt: 